In [26]:
render_dict = ['.', 'P', 'T', '#', 'X']
def render_maze(maze: list[list[float]]) -> list[str]:
    return [''.join([render_dict[y] for y in row]) for row in maze]

In [ ]:
%load_ext autoreload
%autoreload 2

from util.maze_generator import maze_generator
from util.astar_search import a_star_pathfinding
from util.maze_helpers import find_unique_item, GridTile

PATH = 4

maze = maze_generator((8, 14))
path = a_star_pathfinding(maze)

startx, starty = find_unique_item(maze, GridTile.ROBOT.value)
endx, endy = find_unique_item(maze, GridTile.TARGET.value)
for x, y in path:
    if x == startx and y == starty:
        continue
    if x == endx and y == endy:
        continue
    maze[x][y] = PATH

render_maze(maze)

In [ ]:
from stable_baselines3 import PPO
from continuous_env.robot_obstacles import RobotObstacles

env = RobotObstacles(render_mode="human")
model = PPO.load("weights/ppo_model")

obs, _info = env.reset()
avg_reward = 0
num_envs = 1
for _ in range(10000):
    action, _states = model.predict(obs)
    print(action)
    obs, reward, term, trunc, info = env.step(action)
    done = term or trunc
    if done:
        num_envs += 1
        avg_reward += reward
        obs, _info = env.reset()

avg_reward /= num_envs
avg_reward

### testing A star

In [65]:
%load_ext autoreload
%autoreload 2

from util.astar_search import _dijkstras, astar_pathfinding
from util.maze_generator import maze_generator

for _ in range(10):
    maze = maze_generator((8, 14))
    path = astar_pathfinding(maze)
    dist = _dijkstras(maze)

    astar_dist = 0
    prev = path[0]
    for i in path[1:]:
        t = (prev[0] - i[0]) * (prev[1] - i[1])
        astar_dist += (10 if t == 0 else 14)

    assert abs(astar_dist - dist) < 0.0001, f"Difference found, astar ({astar_dist}) versus dijkstras ({dist})"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AssertionError: Difference found, astar (42) versus dijkstras (34)